<a href="https://colab.research.google.com/github/SubramanyaJ/23CS5BSBIS/blob/main/L3_Ant_Colony_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.sum(solution * weights)
    if total_weight > capacity:
        return 0  # Penalize overweight solutions
    return np.sum(solution * values)

def levy_flight(Lambda, size):
    sigma = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
             (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma, size)
    v = np.random.normal(0, 1, size)
    step = u / (np.abs(v) ** (1 / Lambda))
    return step

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cuckoo_search_knapsack(values, weights, capacity, n_nests=25, miter=100, pa=0.25):

    n_items = len(values)
    nests = np.random.randint(0, 2, size=(n_nests, n_items))
    fitness = np.array([knapsack_fitness(n, values, weights, capacity) for n in nests])

    best_idx = np.argmax(fitness)
    best_solution = nests[best_idx].copy()
    best_fitness = fitness[best_idx]

    Lambda = 1.5  # Levy flight exponent

    for iteration in range(miter):
        for i in range(n_nests):
            step = levy_flight(Lambda, n_items)
            current = nests[i].astype(float)
            new_solution_cont = current + step
            probs = sigmoid(new_solution_cont)
            new_solution_bin = (probs > 0.5).astype(int)

            new_fitness = knapsack_fitness(new_solution_bin, values, weights, capacity)

            # Greedy selection
            if new_fitness > fitness[i]:
                nests[i] = new_solution_bin
                fitness[i] = new_fitness

                if new_fitness > best_fitness:
                    best_fitness = new_fitness
                    best_solution = new_solution_bin.copy()

        # Abandon worst nests with probability pa
        n_abandon = int(pa * n_nests)
        if n_abandon > 0:
            abandon_indices = np.random.choice(n_nests, n_abandon, replace=False)
            for idx in abandon_indices:
                nests[idx] = np.random.randint(0, 2, n_items)
                fitness[idx] = knapsack_fitness(nests[idx], values, weights, capacity)

        # Update global best after abandonment
        current_best_idx = np.argmax(fitness)
        if fitness[current_best_idx] > best_fitness:
            best_fitness = fitness[current_best_idx]
            best_solution = nests[current_best_idx].copy()

        # Print progress: every 10 iterations and first iteration
        if iteration == 0 or (iteration + 1) % 10 == 0:
            print(f"Iteration {iteration + 1}/{miter}, Best Fitness: {best_fitness}")

    return best_solution, best_fitness

if __name__ == "__main__":
    # Example knapsack problem
    values = np.array([60, 100, 120, 80, 30])
    weights = np.array([10, 20, 30, 40, 50])
    capacity = 100

    best_sol, best_val = cuckoo_search_knapsack(values, weights, capacity, n_nests=30, miter=100, pa=0.25)

    print("\nBest solution found:")
    print(best_sol)
    print("Total value:", best_val)
    print("Total weight:", np.sum(best_sol * weights))

Iteration 1/100, Best Fitness: 280
Iteration 10/100, Best Fitness: 360
Iteration 20/100, Best Fitness: 360
Iteration 30/100, Best Fitness: 360
Iteration 40/100, Best Fitness: 360
Iteration 50/100, Best Fitness: 360
Iteration 60/100, Best Fitness: 360
Iteration 70/100, Best Fitness: 360
Iteration 80/100, Best Fitness: 360
Iteration 90/100, Best Fitness: 360
Iteration 100/100, Best Fitness: 360

Best solution found:
[1 1 1 1 0]
Total value: 360
Total weight: 100
